In [1]:
from pymongo import MongoClient

client = MongoClient('mongodb:')

db = client.losses
db.list_collection_names()

['windhail']

In [2]:
# get the windhail collection
collection = db.windhail

In [3]:
# compute the AAL by summing the L values in the Loss_Distribution and dividing by 100,000

pipeline = [
    { "$unwind": "$Loss_Distribution"},     # Unwind the array so we now have a record for every array element in every document
    { "$group": {                        
        "_id": "$RiskID",                   # group by RiskID since these are associated to each house
        "AAL": {"$sum": { "$divide": ["$Loss_Distribution.L" , 100000] }}
    } }
]
list(collection.aggregate(pipeline))

[{'_id': 4777030934929053, 'AAL': 0.033086541208339},
 {'_id': 6579297085349254, 'AAL': 220.95972585788715},
 {'_id': 7164787666054031, 'AAL': 117.30445086790196},
 {'_id': 2460348369172233, 'AAL': 5.876571806168558},
 {'_id': 1138428042214348, 'AAL': 15.881973969140065},
 {'_id': 4261871628862533, 'AAL': 104.91400491712659},
 {'_id': 4891964357892648, 'AAL': 143.87515733657708},
 {'_id': 7835305964034430, 'AAL': 15.846509137558938},
 {'_id': 7867323049067380, 'AAL': 2.360923662622028},
 {'_id': 8317176042022910, 'AAL': 3.071640194426157},
 {'_id': 4729069634763448, 'AAL': 72.06704620846246},
 {'_id': 5767258794210834, 'AAL': 376.3194080733586},
 {'_id': 4352261505748056, 'AAL': 0.487037085083048},
 {'_id': 6051543064656686, 'AAL': 1.634253431915715},
 {'_id': 7437852952013206, 'AAL': 32.7375412053812},
 {'_id': 8565455702141067, 'AAL': 83.35185273571832},
 {'_id': 8853906085580822, 'AAL': 28.206692847542776},
 {'_id': 1621065218515820, 'AAL': 2.828137326416051},
 {'_id': 9582710987136

In [4]:
# calculate the total aggregate loss for each YearID: Y

pipeline_2 = [
    {"$unwind": "$Loss_Distribution"},
    {"$group": {
        "_id": "$Loss_Distribution.Y", 
        "Total": { "$sum": "$Loss_Distribution.L" }
        }
    }
]

list(collection.aggregate(pipeline_2))[:20]

[{'_id': 36811, 'Total': 11889.6769270841},
 {'_id': 2580, 'Total': 3034.0020748982},
 {'_id': 83034, 'Total': 3044.6526284816},
 {'_id': 87800, 'Total': 3225.1189155578},
 {'_id': 17316, 'Total': 9812.7497943908},
 {'_id': 19158, 'Total': 4998.4632240157},
 {'_id': 58048, 'Total': 33049.167327672},
 {'_id': 32544, 'Total': 4873.6552608029},
 {'_id': 60609, 'Total': 40808.0899383319},
 {'_id': 47570, 'Total': 585.8876366686},
 {'_id': 4830, 'Total': 12461.5177251708},
 {'_id': 86191, 'Total': 3511.2572374925},
 {'_id': 10596, 'Total': 652.4651766532},
 {'_id': 98941, 'Total': 4745.3214702729},
 {'_id': 80898, 'Total': 20803.0304377714},
 {'_id': 20129, 'Total': 233.0722334236},
 {'_id': 6972, 'Total': 6977.2110478474},
 {'_id': 8539, 'Total': 4893.3846541983},
 {'_id': 56617, 'Total': 4072.2967704177},
 {'_id': 51226, 'Total': 18598.9989183147}]

In [5]:
# Find the single EventID which causes the largest aggregate loss on the collection of homes

pipeline_3 = [
    {"$unwind": "$Loss_Distribution"},
    {"$group": {
        "_id": "$Loss_Distribution.E",                     # group by eventID
        "Total": { "$sum": "$Loss_Distribution.L" }        # sum the losses
        }
    },
    {"$sort": {"Total" : -1}},
    {"$limit" : 1}                                         # Limit by 1 to get EventID where maximum loss occured
]

list(collection.aggregate(pipeline_3))

[{'_id': 8923867, 'Total': 4151824.843802035}]